In [30]:
import pandas as pd
import re
import os

In [31]:
input_path = "C:\\Users\\ashle\\Desktop\\Korea-ggumim\\Datasets\\Korea-ggumim.csv"
output_path = "C:\\Users\\ashle\\Desktop\\Korea-ggumim\\Datasets\\cleaned_data.csv"
garbage_dir = "C:\\Users\\ashle\\Desktop\\Korea-ggumim\\Datasets\\Garbage"

In [32]:
os.makedirs(garbage_dir, exist_ok=True)

In [33]:
df = pd.read_csv(input_path)

C:\Users\ashle\AppData\Local\Temp\ipykernel_36856\1758342947.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path)


In [42]:
def validate_and_clean_email(email):
    """
    Validates and cleans an email address:
    - Ensures it's valid using regex.
    - Removes extra dots and @ symbols.
    """
    email_pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'  # Regex to validate email
    if not re.match(email_pattern, email):
        with open(os.path.join(garbage_dir, "invalid_emails.txt"), "a") as f:
            f.write(email + "\n")
        return None  # Invalid email
    return cleaned_email

In [43]:
def clean_nickname(nickname):
    """
    Cleans a nickname:
    - Removes control characters.
    - Allows common symbols like ., _, and -.
    """
    # Remove control characters
    cleaned_nickname = re.sub(r'[\x00-\x1F\x7F]', '', nickname)

    # Log nicknames with control characters to garbage file
    if nickname != cleaned_nickname:
        with open(os.path.join(garbage_dir, "invalid_nicknames.txt"), "a") as f:
            f.write(nickname + "\n")

    return cleaned_nickname

In [37]:
# Function to remove duplicates
def remove_duplicates(df):
    global garbage
    duplicates = df[df.duplicated()]  # Identify duplicates
    garbage = pd.concat([garbage, duplicates])  # Store duplicate rows
    df.drop_duplicates(inplace=True)
    print(f"Duplicates removed. Remaining rows: {df.shape[0]}")
    return df


In [46]:
# Define the validation and cleaning functions
def validate_and_clean_email(email):
    """
    Validates and cleans an email address:
    - Ensures it's valid using regex.
    - Removes extra dots and @ symbols.
    """
    import re
    email_pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'  # Regex to validate email
    if not re.match(email_pattern, email):
        return None  # Invalid email

    # Clean up extra dots and @ symbols
    cleaned_email = email.replace("..", ".").replace("@@", "@")

    # Revalidate after cleaning
    return cleaned_email if re.match(email_pattern, cleaned_email) else None


def clean_nickname(nickname):
    """
    Cleans a nickname:
    - Removes control characters.
    - Allows common symbols like ., _, and -.
    """
    import re
    # Remove control characters
    cleaned_nickname = re.sub(r'[\x00-\x1F\x7F]', '', nickname)
    return cleaned_nickname

# Ensure you define the functions above before applying them to the DataFrame


In [18]:
# Function to remove outliers
def remove_outliers(df):
    global garbage
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    for col in numerical_cols:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        garbage = pd.concat([garbage, outliers])  # Store outliers
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]  # Keep valid data
    print(f"Outliers removed. Remaining rows: {df.shape[0]}")
    return df


In [47]:
# Save the cleaned data to a new CSV
df.to_csv(output_path, index=False)

# Merge garbage files into one
merged_garbage_file = os.path.join(garbage_dir, "merged_garbage.txt")
with open(merged_garbage_file, "w") as outfile:
    for garbage_file in os.listdir(garbage_dir):
        if garbage_file.endswith(".txt") or garbage_file.endswith(".csv"):
            with open(os.path.join(garbage_dir, garbage_file), "r") as infile:
                outfile.write(f"\n--- {garbage_file} ---\n")
                outfile.write(infile.read())

print(f"Cleaning complete! Cleaned data saved to '{output_path}'")
print(f"Garbage files merged into '{merged_garbage_file}'")

Cleaning complete! Cleaned data saved to 'C:\Users\ashle\Desktop\Korea-ggumim\Datasets\cleaned_data.csv'
Garbage files merged into 'C:\Users\ashle\Desktop\Korea-ggumim\Datasets\Garbage\merged_garbage.txt'


In [52]:
# Define README content
readme_content = """
# Korea Ggumim Data Cleaning Project

This project involves cleaning a dataset containing user information from Korea Ggumim. The tasks performed include:

1. Cleaning and validating email addresses.
2. Cleaning nicknames to remove control characters while retaining common symbols.

## Files in the Project

- `raw_data.csv`: The original dataset.
- `cleaned_data.csv`: The cleaned dataset.
- `garbage_files`: Temporary files created during processing.

## Steps in the Cleaning Process

1. Validate email addresses:
    - Remove invalid emails.
    - Strip extra dots and symbols.
2. Clean nicknames:
    - Remove control characters.
    - Allow common symbols like `.`, `_`, and `-`.

## How to Run

1. Ensure all dependencies are installed:
    ```
    pip install pandas numpy
    ```
2. Run the main script:
    ```
    python clean_data.py
    ```
3. The cleaned dataset will be saved as `cleaned_data.csv`.

## Garbage Files

Temporary files generated during cleaning can be found in the `garbage_files/` directory. These can be merged or deleted after the process.

## Notes

- Ensure the input file is named `Korea-ggumim.csv` and placed in the `Datasets/` directory.
"""

# Save README content to a file
with open("README.md", "w") as readme_file:
    readme_file.write(readme_content)

print("README.md created successfully!")



README.md created successfully!
